# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data_path = "../WeatherPy/output_data/cities.csv"

cities_data = pd.read_csv(cities_data_path)

cities_data.head()

,Unnamed: 0,City,Lats,Lngs,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Nikolskoye,59.7035,30.7861,18.77,82,49,1.17,RU,1659555815
1,1,Busselton,-33.6500,115.3333,11.89,73,95,8.35,AU,1659555816
2,2,Mataura,-46.1927,168.8643,6.93,87,100,1.84,NZ,1659555816
3,3,Olafsvik,64.8945,-23.7142,10.02,65,15,5.07,IS,1659555601
4,4,Rikitea,-23.1203,-134.9692,22.29,68,71,7.16,PF,1659555817


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [1]:
#enable the gmaps extension
#!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [10]:
#configure gmaps
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = cities_data[["Lats", "Lngs"]]

#store humidity in humidity
humidity= cities_data["Humidity"]

In [11]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#filtering weather by celsius
filter_weather = cities_data.loc[(cities_data["Max Temp"] < 26.6667) & (cities_data["Max Temp"] > 21.1111)]

#filtering our data again to have wind speed < 10
filter_weather = filter_weather.loc[filter_weather["Wind Speed"] < 10]

#and again so that we have clear skies
filter_weather = filter_weather.loc[filter_weather["Cloudiness"] == 0]

#display data
#there are no null values
filter_weather

,Unnamed: 0,City,Lats,Lngs,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
259,259,Lamu,-2.2717,40.9020,24.02,78,0,7.40,KE,1659555943
310,310,Hede,33.7681,120.2517,25.52,94,0,2.08,CN,1659555968
371,371,Birjand,32.8663,59.2211,23.26,20,0,3.09,IR,1659555997
378,378,Morondava,-20.2833,44.2833,22.74,83,0,3.20,MG,1659556000
428,428,Port Angeles,48.1181,-123.4307,21.47,76,0,2.68,US,1659555535
481,481,Arona,28.0996,-16.6810,22.61,69,0,8.75,ES,1659556051
531,531,Milas,37.3164,27.7839,24.64,73,0,1.54,TR,1659556077


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#rename dataframe
hotel_df = filter_weather 

#setting up the url construction
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"radius": 5000,
    "type": "lodging",
    "key": g_key}

for index, row in hotel_df.iterrows():
    
    #get coords from df
    lat = row["Lats"]
    lng = row["Lngs"]
    
    #puts our lat and lngs into our location parameter, this is inside our loop because it is changing
    params["location"] = f"{lat},{lng}"
    
    #get results
    print(f"Retrieving results for index {index} : {row['City']}")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        #printing our results
        print(f"The nearest hotel is {results[0]['name']}")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        #printing our result if we dont have a hotel within 5000m
        print("Missing field/result... skipping.")
    print("-------------")


Retrieving results for index 259 : Lamu
The nearest hotel is Lamu House
-------------
Retrieving results for index 310 : Hede
The nearest hotel is Vienna 3 Best Hotel Sheyang Jiefang Rd
-------------
Retrieving results for index 371 : Birjand
The nearest hotel is Birjand Tourist Hotel
-------------
Retrieving results for index 378 : Morondava
The nearest hotel is La Case Bambou
-------------
Retrieving results for index 428 : Port Angeles
The nearest hotel is Red Lion Hotel Port Angeles Harbor
-------------
Retrieving results for index 481 : Arona
The nearest hotel is Paradise Court
-------------
Retrieving results for index 531 : Milas
The nearest hotel is Milashan Hotel
-------------


In [7]:
#use this to remove a city that did not have a hotel within 5000 meters  
#clean_hotel_df = hotel_df.loc[hotel_df["City"] != "city name"]
clean_hotel_df = hotel_df

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Lats", "Lngs"]]
markers = gmaps.marker_layer(locations)

In [9]:
# Create hotel symbol layer

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))